In [2]:
import random
import string


# 1. Przyjmij następujący zbiór danych wejściowych:

In [42]:
data1 = "bbb$"
data2 = "aabbabd"
data3 = "ababcd"
data4 = "abaababaabaabaabab$"
data5 = ''.join(random.choices(string.ascii_lowercase, k=100))
data6 = None

with open('1997_714_head.txt', 'r') as file:
    data6 = file.read()

# 2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.

In [43]:
def marker_check(text, marker='$'):
    lines = text.splitlines()
    processed = []
    
    for line in lines:
        if not line.endswith(marker):
            line += marker
        if marker in line: # jesli ma go w środku, to zostaje usunięty
            line = line.replace(marker, '')
        processed.append(line)
    
    return '\n'.join(processed)

# 3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

In [55]:
class TrieNode:
    def __init__(self):
        self.children = {}


class SuffixTrie:
    def __init__(self, text):
        self.root = TrieNode()
        self.construct_(text)

    def insert_(self, text, suffix):
        node = self.root
        for char in suffix:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]

    def construct_(self, text):
        n = len(text)
        for i in range(n):
            suffix = text[i:]
            self.insert_(text, suffix)

    def search_(self, suffix):
        node = self.root
        for char in suffix:
            if char not in node.children:
                return False
            node = node.children[char]
        return True
    
    def get_leaf_indices(self, node):
        indices = []
        if not node.children:
            indices.append(node.index)
        for child in node.children.values():
            indices.extend(self.get_leaf_indices(child))
        return indices

    def get_occurrences(self, suffix):
        node = self.root
        for char in suffix:
            if char not in node.children:
                return []
            node = node.children[char]
        return self.get_leaf_indices(node)

# 4. Zaimplementuj algorytm konstruujący drzewo sufiksów.

In [56]:
class SuffixTreeNode:
    def __init__(self, char):
        self.char = char
        self.children = {}
        self.indices = []


class SuffixTree:
    def __init__(self, text):
        self.root = SuffixTreeNode('')
        self.construct_(text)
        
    def insert_(self, suffix, index):
        current_node = self.root
        for char in suffix:
            if char not in current_node.children:
                current_node.children[char] = SuffixTreeNode(char)
            current_node = current_node.children[char]
        current_node.indices.append(index)

    def construct_(self, text):
        n = len(text)
        for i in range(n):
            self.insert_(text[i:], i)

    def search_(self, suffix):
        current_node = self.root
        for char in suffix:
            if char not in current_node.children:
                return False
            current_node = current_node.children[char]
        return True

    def get_leaf_indices(self, node):
        indices = node.indices.copy()
        for child in node.children.values():
            indices.extend(self.get_leaf_indices(child))
        return indices

    def get_occurrences(self, suffix):
        current_node = self.root
        for char in suffix:
            if char not in current_node.children:
                return []
            current_node = current_node.children[char]
        return self.get_leaf_indices(current_node)

# 5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.

Algorytm wyszukiwania wzorców (Algorytm Knutha-Morrisa-Pratta) został wzięty z laboratorium pierwszego.

In [57]:
def prefix_function(pattern):
    m = len(pattern)
    pi = [0] * m
    k = 0

    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k

    return pi


def KMP(text, pattern):
    n = len(text)
    m = len(pattern)
    pi = prefix_function(pattern)
    q = 0
    result = []

    for i in range(n):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]
        if pattern[q] == text[i]:
            q += 1
        if q == m:
            result.append(i - m + 1)
            q = pi[q-1]

    return result

Funkcja testująca:

In [59]:
def test(text, pattern):
    suffix_tree = SuffixTree(text)
    trie = SuffixTrie(text)
    suffix_tree_occurrences = suffix_tree.get_occurrences(pattern)
    trie_occurrences = trie.get_occurrences(pattern)
    kmp_occurrences = KMP(text, pattern)
    if sorted(suffix_tree_occurrences) == sorted(kmp_occurrences) and sorted(kmp_occurrences) == sorted(trie_occurrences):
        print("Wyniki są zgodne!")
    else:
        print("Wyniki nie są zgodne!")
        
test("bananasbananas", "ananas")

AttributeError: 'TrieNode' object has no attribute 'index'